In [2]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping

import tensorflow as tf

seed = 777
np.random.seed(seed)
tf.set_random_seed(seed)

Using TensorFlow backend.


In [3]:
df = pd.read_csv('data/Train_1.csv', encoding= 'ansi')

In [4]:
df.shape

(6025, 72)

In [5]:
x_row = df.iloc[:, :-1]
y_row = df.iloc[:, -1:]
x_row.shape

(6025, 71)

In [6]:
x_row.head()

,총_유동인구_수,남성_유동인구_수,여성_유동인구_수,연령대_10_유동인구_수,연령대_20_유동인구_수,연령대_30_유동인구_수,연령대_40_유동인구_수,연령대_50_유동인구_수,연령대_60_이상_유동인구_수,총_상주인구_수,...,여성_매출_금액,주중_매출_건수,주말_매출_건수,남성_매출_건수,여성_매출_건수,점포수,상권_변화_지표_HH,상권_변화_지표_HL,상권_변화_지표_LH,상권_변화_지표_LL
0,173660,96866,76793,6356,52613,39480,32053,28652,14507,1314,...,1.600597e+09,141199,67904,118723,84751,110,0,0,0,1
1,209238,107003,102237,11228,45589,47885,44694,40624,19212,2514,...,6.824956e+08,60820,29820,50400,31521,63,1,0,0,0
2,478882,243693,235189,24111,83630,112851,109472,97958,50859,3743,...,3.451869e+09,285521,107030,215844,161208,219,0,0,0,1
3,1078154,556415,521736,63220,353808,235924,168800,157170,99228,1577,...,3.800171e+10,2571027,1394990,1784205,2117802,712,0,0,1,0
4,79702,48450,31256,3792,32157,16778,13658,10164,3155,578,...,1.687361e+09,264219,81132,186809,141214,110,0,0,1,0


In [7]:
x_row_nostd = x_row.iloc[:, -4:].values
x_row_std = x_row.iloc[:, :-4].values
x_row_nostd[:1]

array([[0, 0, 0, 1]], dtype=int64)

In [8]:
# 회귀 예측은 Standard, 분류 예측은 MinMax

In [9]:
scaler = preprocessing.StandardScaler()

In [10]:
x_std = scaler.fit_transform(x_row_std)
#y_std = scaler.fit_transform(y_row)

In [11]:
y_std = np.log1p(y_row.values)
y_std[0]

array([3.93573953])

In [12]:
x_str = np.concatenate((x_std, x_row_nostd), axis= 1)

In [13]:
x_str[0, -4:]

array([0., 0., 0., 1.])

In [14]:
X_train, X_test, y_train, y_test = train_test_split(x_str, y_std, 
                                                    test_size= 0.2, random_state= seed)

In [15]:
print('X_train.shape : ', X_train.shape, 'y_train.shape : ', y_train.shape)
print('X_test.shape : ', X_test.shape, 'y_test.shape : ', y_test.shape)

X_train.shape :  (4820, 71) y_train.shape :  (4820, 1)
X_test.shape :  (1205, 71) y_test.shape :  (1205, 1)


In [18]:
model = Sequential()
model.add(Dense(512, input_dim= x_row.shape[1], activation= 'relu'))
model.add(Dense(256, activation= 'relu'))
model.add(Dense(128, activation= 'relu'))
model.add(Dense(64, activation= 'relu'))
model.add(Dense(8, activation= 'relu'))
model.add(Dense(1))

model.compile(loss= 'mean_squared_error',
              optimizer= 'adam',
              metrics= ['accuracy'])

early_stopping_callback = EarlyStopping(monitor= 'val_loss', patience= 100)

model.fit(X_train, y_train, validation_split= 0.15,
          epochs= 3000, batch_size= 300, verbose= 1,
          callbacks= [early_stopping_callback])

Train on 4097 samples, validate on 723 samples
Epoch 1/3000
4097/4097 [==============================] - 1s 127us/step - loss: 9.3535 - accuracy: 0.0000e+00 - val_loss: 2.8035 - val_accuracy: 0.0000e+00
Epoch 2/3000
4097/4097 [==============================] - 0s 31us/step - loss: 1.8721 - accuracy: 0.0000e+00 - val_loss: 1.4904 - val_accuracy: 0.0000e+00
Epoch 3/3000
4097/4097 [==============================] - 0s 30us/step - loss: 1.0663 - accuracy: 0.0000e+00 - val_loss: 1.0111 - val_accuracy: 0.0000e+00
Epoch 4/3000
4097/4097 [==============================] - 0s 30us/step - loss: 0.7074 - accuracy: 0.0000e+00 - val_loss: 0.7295 - val_accuracy: 0.0000e+00
Epoch 5/3000
4097/4097 [==============================] - 0s 30us/step - loss: 0.5299 - accuracy: 0.0000e+00 - val_loss: 0.5140 - val_accuracy: 0.0000e+00
Epoch 6/3000
4097/4097 [==============================] - 0s 30us/step - loss: 0.3902 - accuracy: 0.0000e+00 - val_loss: 0.3838 - val_accuracy: 0.0000e+00
Epoch 7/3000
4097/4097

4097/4097 [==============================] - 0s 30us/step - loss: 0.0258 - accuracy: 0.0000e+00 - val_loss: 0.0361 - val_accuracy: 0.0000e+00
Epoch 54/3000
4097/4097 [==============================] - 0s 29us/step - loss: 0.0223 - accuracy: 0.0000e+00 - val_loss: 0.0433 - val_accuracy: 0.0000e+00
Epoch 55/3000
4097/4097 [==============================] - 0s 30us/step - loss: 0.0280 - accuracy: 0.0000e+00 - val_loss: 0.0404 - val_accuracy: 0.0000e+00
Epoch 56/3000
4097/4097 [==============================] - 0s 29us/step - loss: 0.0253 - accuracy: 0.0000e+00 - val_loss: 0.0424 - val_accuracy: 0.0000e+00
Epoch 57/3000
4097/4097 [==============================] - 0s 30us/step - loss: 0.0231 - accuracy: 0.0000e+00 - val_loss: 0.0388 - val_accuracy: 0.0000e+00
Epoch 58/3000
4097/4097 [==============================] - 0s 29us/step - loss: 0.0233 - accuracy: 0.0000e+00 - val_loss: 0.0348 - val_accuracy: 0.0000e+00
Epoch 59/3000
4097/4097 [==============================] - 0s 29us/step - loss

4097/4097 [==============================] - 0s 30us/step - loss: 0.0100 - accuracy: 0.0000e+00 - val_loss: 0.0307 - val_accuracy: 0.0000e+00
Epoch 106/3000
4097/4097 [==============================] - 0s 30us/step - loss: 0.0143 - accuracy: 0.0000e+00 - val_loss: 0.0242 - val_accuracy: 0.0000e+00
Epoch 107/3000
4097/4097 [==============================] - 0s 29us/step - loss: 0.0090 - accuracy: 0.0000e+00 - val_loss: 0.0225 - val_accuracy: 0.0000e+00
Epoch 108/3000
4097/4097 [==============================] - 0s 30us/step - loss: 0.0078 - accuracy: 0.0000e+00 - val_loss: 0.0206 - val_accuracy: 0.0000e+00
Epoch 109/3000
4097/4097 [==============================] - 0s 30us/step - loss: 0.0072 - accuracy: 0.0000e+00 - val_loss: 0.0216 - val_accuracy: 0.0000e+00
Epoch 110/3000
4097/4097 [==============================] - 0s 30us/step - loss: 0.0082 - accuracy: 0.0000e+00 - val_loss: 0.0267 - val_accuracy: 0.0000e+00
Epoch 111/3000
4097/4097 [==============================] - 0s 31us/step 

4097/4097 [==============================] - 0s 30us/step - loss: 0.0065 - accuracy: 0.0000e+00 - val_loss: 0.0211 - val_accuracy: 0.0000e+00
Epoch 158/3000
4097/4097 [==============================] - 0s 30us/step - loss: 0.0059 - accuracy: 0.0000e+00 - val_loss: 0.0193 - val_accuracy: 0.0000e+00
Epoch 159/3000
4097/4097 [==============================] - 0s 31us/step - loss: 0.0051 - accuracy: 0.0000e+00 - val_loss: 0.0170 - val_accuracy: 0.0000e+00
Epoch 160/3000
4097/4097 [==============================] - 0s 30us/step - loss: 0.0081 - accuracy: 0.0000e+00 - val_loss: 0.0193 - val_accuracy: 0.0000e+00
Epoch 161/3000
4097/4097 [==============================] - 0s 31us/step - loss: 0.0089 - accuracy: 0.0000e+00 - val_loss: 0.0217 - val_accuracy: 0.0000e+00
Epoch 162/3000
4097/4097 [==============================] - 0s 30us/step - loss: 0.0078 - accuracy: 0.0000e+00 - val_loss: 0.0211 - val_accuracy: 0.0000e+00
Epoch 163/3000
4097/4097 [==============================] - 0s 30us/step 

4097/4097 [==============================] - 0s 29us/step - loss: 0.0052 - accuracy: 0.0000e+00 - val_loss: 0.0193 - val_accuracy: 0.0000e+00
Epoch 210/3000
4097/4097 [==============================] - 0s 29us/step - loss: 0.0067 - accuracy: 0.0000e+00 - val_loss: 0.0198 - val_accuracy: 0.0000e+00
Epoch 211/3000
4097/4097 [==============================] - 0s 28us/step - loss: 0.0063 - accuracy: 0.0000e+00 - val_loss: 0.0165 - val_accuracy: 0.0000e+00
Epoch 212/3000
4097/4097 [==============================] - 0s 29us/step - loss: 0.0064 - accuracy: 0.0000e+00 - val_loss: 0.0277 - val_accuracy: 0.0000e+00
Epoch 213/3000
4097/4097 [==============================] - 0s 31us/step - loss: 0.0077 - accuracy: 0.0000e+00 - val_loss: 0.0204 - val_accuracy: 0.0000e+00
Epoch 214/3000
4097/4097 [==============================] - 0s 30us/step - loss: 0.0039 - accuracy: 0.0000e+00 - val_loss: 0.0163 - val_accuracy: 0.0000e+00
Epoch 215/3000
4097/4097 [==============================] - 0s 30us/step 

4097/4097 [==============================] - 0s 31us/step - loss: 0.0017 - accuracy: 0.0000e+00 - val_loss: 0.0151 - val_accuracy: 0.0000e+00
Epoch 262/3000
4097/4097 [==============================] - 0s 30us/step - loss: 0.0015 - accuracy: 0.0000e+00 - val_loss: 0.0151 - val_accuracy: 0.0000e+00
Epoch 263/3000
4097/4097 [==============================] - 0s 30us/step - loss: 0.0015 - accuracy: 0.0000e+00 - val_loss: 0.0162 - val_accuracy: 0.0000e+00
Epoch 264/3000
4097/4097 [==============================] - 0s 31us/step - loss: 0.0018 - accuracy: 0.0000e+00 - val_loss: 0.0154 - val_accuracy: 0.0000e+00
Epoch 265/3000
4097/4097 [==============================] - 0s 30us/step - loss: 0.0025 - accuracy: 0.0000e+00 - val_loss: 0.0154 - val_accuracy: 0.0000e+00
Epoch 266/3000
4097/4097 [==============================] - 0s 30us/step - loss: 0.0037 - accuracy: 0.0000e+00 - val_loss: 0.0165 - val_accuracy: 0.0000e+00
Epoch 267/3000
4097/4097 [==============================] - 0s 29us/step 

4097/4097 [==============================] - 0s 28us/step - loss: 0.0015 - accuracy: 0.0000e+00 - val_loss: 0.0149 - val_accuracy: 0.0000e+00
Epoch 314/3000
4097/4097 [==============================] - 0s 29us/step - loss: 0.0016 - accuracy: 0.0000e+00 - val_loss: 0.0153 - val_accuracy: 0.0000e+00
Epoch 315/3000
4097/4097 [==============================] - 0s 28us/step - loss: 0.0015 - accuracy: 0.0000e+00 - val_loss: 0.0151 - val_accuracy: 0.0000e+00
Epoch 316/3000
4097/4097 [==============================] - 0s 28us/step - loss: 0.0018 - accuracy: 0.0000e+00 - val_loss: 0.0185 - val_accuracy: 0.0000e+00
Epoch 317/3000
4097/4097 [==============================] - 0s 29us/step - loss: 0.0040 - accuracy: 0.0000e+00 - val_loss: 0.0194 - val_accuracy: 0.0000e+00
Epoch 318/3000
4097/4097 [==============================] - 0s 29us/step - loss: 0.0032 - accuracy: 0.0000e+00 - val_loss: 0.0186 - val_accuracy: 0.0000e+00
Epoch 319/3000
4097/4097 [==============================] - 0s 29us/step 

4097/4097 [==============================] - 0s 31us/step - loss: 0.0039 - accuracy: 0.0000e+00 - val_loss: 0.0164 - val_accuracy: 0.0000e+00
Epoch 366/3000
4097/4097 [==============================] - 0s 28us/step - loss: 0.0047 - accuracy: 0.0000e+00 - val_loss: 0.0179 - val_accuracy: 0.0000e+00
Epoch 367/3000
4097/4097 [==============================] - 0s 29us/step - loss: 0.0038 - accuracy: 0.0000e+00 - val_loss: 0.0153 - val_accuracy: 0.0000e+00
Epoch 368/3000
4097/4097 [==============================] - 0s 32us/step - loss: 0.0025 - accuracy: 0.0000e+00 - val_loss: 0.0150 - val_accuracy: 0.0000e+00
Epoch 369/3000
4097/4097 [==============================] - 0s 31us/step - loss: 0.0017 - accuracy: 0.0000e+00 - val_loss: 0.0142 - val_accuracy: 0.0000e+00
Epoch 370/3000
4097/4097 [==============================] - 0s 29us/step - loss: 0.0013 - accuracy: 0.0000e+00 - val_loss: 0.0152 - val_accuracy: 0.0000e+00
Epoch 371/3000
4097/4097 [==============================] - 0s 29us/step 

KeyboardInterrupt: 

In [16]:
#test = pd.read_csv('../Final_Test_v6.csv', encoding= 'ansi')
#X_test = test.iloc[:, :-1]
#y_test = test.iloc[:, -1:]
#y_test.head()

In [17]:
#X_test_std = scaler.fit_transform(X_test)
#y_test_std = scaler.fit_transform(y_test)

In [18]:
Y_prediction = model.predict(X_test)

In [19]:
from sklearn.metrics import r2_score

In [20]:
r2 = r2_score(y_test, Y_prediction)
r2

0.9671469953487621

In [21]:
Y_prediction_ = np.expm1(Y_prediction)
Y_prediction_[:5]

array([[27.32325 ],
       [23.254234],
       [58.375282],
       [65.707085],
       [44.396378]], dtype=float32)

In [22]:
#for i in range(100):
  #  label = y_test_[i, 0]
 #   prediction = Y_prediction_[i, 0]
 #   print('step :', i, '==> Real : {:6.3f}\t====> pred : {:6.3f}\terror : ====> {:6.3f}'.
 #         format(label, prediction, label - prediction))

In [23]:
y_test_ = np.expm1(y_test)

In [24]:
Y_pred = pd.DataFrame(Y_prediction_)
Y_real = pd.DataFrame(y_test_)

In [25]:
result = pd.DataFrame()
result['Y_pred'] = Y_pred.values.flatten()
result['Y_real'] = Y_real.values.flatten()
result['Mean'] = result['Y_pred'] - result['Y_real']
result['Mean'] = result['Mean'].abs()
result.describe()

,Y_pred,Y_real,Mean
count,1205.000000,1205.000000,1205.000000
mean,48.642666,48.844149,2.588865
std,37.085392,37.185880,5.311975
min,18.587309,20.900000,0.005111
25%,29.954620,30.400000,0.498239
50%,42.178860,41.500000,1.260178
75%,54.403049,54.900000,2.582980
max,346.237152,295.100000,75.337152


In [26]:
#result.to_csv('result_.csv')

In [27]:
X_test = pd.read_csv('/Test_1.csv', encoding= 'ansi')

In [29]:
X_test.head()

,총_유동인구_수,남성_유동인구_수,여성_유동인구_수,연령대_10_유동인구_수,연령대_20_유동인구_수,연령대_30_유동인구_수,연령대_40_유동인구_수,연령대_50_유동인구_수,연령대_60_이상_유동인구_수,총_상주인구_수,...,여성_매출_금액,주중_매출_건수,주말_매출_건수,남성_매출_건수,여성_매출_건수,점포수,상권_변화_지표_HH,상권_변화_지표_HL,상권_변화_지표_LH,상권_변화_지표_LL
0,173660,96866,76793,6356,52613,39480,32053,28652,14507,1314,...,1.600597e+09,141199,67904,118723,84751,110,0,0,0,1
1,209238,107003,102237,11228,45589,47885,44694,40624,19212,2514,...,6.824956e+08,60820,29820,50400,31521,63,1,0,0,0
2,478882,243693,235189,24111,83630,112851,109472,97958,50859,3743,...,3.451869e+09,285521,107030,215844,161208,219,0,0,0,1
3,1078154,556415,521736,63220,353808,235924,168800,157170,99228,1577,...,3.800171e+10,2571027,1394990,1784205,2117802,712,0,0,1,0
4,79702,48450,31256,3792,32157,16778,13658,10164,3155,578,...,1.687361e+09,264219,81132,186809,141214,110,0,0,1,0


In [30]:
X_test_nostd = X_test.iloc[:, -4:].values
X_test_std = X_test.iloc[:, :-4].values
X_test_nostd[:1]

array([[0, 0, 0, 1]], dtype=int64)

In [31]:
scaler = preprocessing.StandardScaler()
x_std = scaler.fit_transform(X_test_std)

In [34]:
x_str = np.concatenate((x_std, X_test_nostd), axis= 1)
x_str[0, -4:]

array([0., 0., 0., 1.])

In [35]:
Y_prediction = model.predict(x_str)

In [36]:
Y_prediction_ = np.expm1(Y_prediction)

In [37]:
Y_pred = pd.DataFrame(Y_prediction_)
Y_pred.head()

,0
0,48.459084
1,46.822044
2,42.880836
3,58.093201
4,54.271984


In [38]:
Y_pred.to_csv('Test_result.csv', encoding= 'ansi', index= False)